In [564]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
!pip install selenium
!pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

In [565]:
# Grab the Intial Page showing the First Group of Comapnies
page = requests.get("https://canada.constructconnect.com/dcn/construction-directory/search?ccpage=1&perpage=16&phrase&category&location#results").text

In [566]:
# Parse the Page and create BS Object
soup = BeautifulSoup(page, "html.parser")

In [567]:
# Find the Section that has the listed Companies on the Page
allSections = soup.find_all('section')
for index, section in enumerate(allSections):
    if("Search Results" in str(section)):
        find_section = index
print(find_section)

4


In [568]:
# Find the Links to each Company
links = allSections[find_section].find_all('a')
links[0:5]

[<a class="promos-cover promos-cover--visible" href="/dcn/construction-directory/22005C29-8865-4FAD-B82C-46A622ED5EFD" title="ConstructConnect Details"></a>,
 <a class="link-tertiary" href="/dcn/construction-directory/22005C29-8865-4FAD-B82C-46A622ED5EFD" title="ConstructConnect Details">See company details<i class="icon icon-angle--right"></i></a>,
 <a class="promos-cover promos-cover--visible" href="/dcn/construction-directory/6B2F4E69-B21F-4B85-AF84-FC02CB71CBDC" title="#1 Prime Contracting Details"></a>,
 <a class="link-tertiary" href="/dcn/construction-directory/6B2F4E69-B21F-4B85-AF84-FC02CB71CBDC" title="#1 Prime Contracting Details">See company details<i class="icon icon-angle--right"></i></a>,
 <a class="promos-cover promos-cover--visible" href="/dcn/construction-directory/AEADEAF6-1E99-462C-8605-B0B462E7DD60" title="1187839 Ontario Ltd Details"></a>]

In [569]:
# There are extra links and Duplicates, so exclude the one's with HTTP and get rid of deuplicates
hrefs = []
for link in allSections[find_section].find_all('a'):
    if ('http' not in link):
        hrefs.append(link.get('href'))

# Get Rid of the Duplicates
hrefs = [*set(hrefs)]
# Get Rid of 'http' hrefs
hrefs = [ x for x in z if "http" not in x ]
hrefs[2]

'/dcn/construction-directory/d3e2a282-342e-4074-a873-a5fc54ee244f'

In [891]:
# A Function to Clean the data for the single company
def clean_data(initial_data):
    
    # get rid of the \n and strip blank spaces
    initial_data = [x.replace('\n', '') for x in initial_data]
    initial_data = [elem.strip() for elem in new_set]
    
    # Split the Address, strip the spaces    
    address = initial_data[1]
    split_list = address.split(',')
    [i.strip() for i in split_list]

    # Split the City and Zip Code and then delete the spaces
    a = split_list[2]
    a = re.split(r'\s{4,}', a)
    del a[0]

    # Delete the old Addresses
    del split_list[2]
    del initial_data[1]

    # Add the data back together and strip address space 
    initial_data = initial_data + split_list + a
    initial_data = [i.strip() for i in initial_data]
    
    return initial_data

In [965]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

# Will Create Loop of ALL Pages to then Loop Companies on each Page
# Will Change to Loop of Companies on Single Page

data = pd.DataFrame()

for i,url in enumerate(hrefs):
    driver.get('https://canada.constructconnect.com/dcn/construction-directory/search?ccpage=1&perpage=16&phrase&category&location#results')

    driver.find_element("xpath", f"//a[contains(@href, '{hrefs[i]}')]").click()

    # Get the Contact/Address info for Company (I used BeautifulSoup becasue it is more familiar to me)
    page = requests.get("https://canada.constructconnect.com" + hrefs[i])
    soup2 = BeautifulSoup(page.text)
    
    for ulTag in soup2.find_all('ul', {'class': 'company-details'}):
        liTag = ulTag.find_all("li")
        if (liTag != []):
            data.append(liTag)
    
    if len(liTag) == 4:
        work_org = liTag[0].text.strip()
        type_of_work = liTag[1].text.strip()     
        address = liTag[2].text.strip()
        phone_number = liTag[3].text.strip()  
        data = data.append({"Work_Org": work_org, "Type_of_Work": type_of_work, "Address": address, "Phone_Number": phone_number}, ignore_index=True)
 
    if len(liTag) == 5:
        work_org = liTag[0].text.strip()
        type_of_work = liTag[1].text.strip()     
        address = liTag[2].text.strip()
        phone_number = liTag[3].text.strip()  
        website = liTag[4].text.strip()
        data = data.append({"Work_Org": work_org, "Type_of_Work": type_of_work, "Address": address, "Phone_Number": phone_number, "Website": website}, ignore_index=True)
    
    
    if len(liTag) == 6:
        work_org = liTag[0].text.strip()
        type_of_work = liTag[1].text.strip()     
        address = liTag[2].text.strip()
        phone_number = liTag[3].text.strip()  
        website = liTag[4].text.strip()
        email = liTag[5].text.strip()
        data = data.append({"Work_Org": work_org, "Type_of_Work": type_of_work, "Address": address, "Phone_Number": phone_number, "Website": website, "Email": email}, ignore_index=True)
    
    # Call Function to clean the data



    # Get Company Name
#     company_name = soup2.select('h1.company-name')[0].text.strip()


C:\Users\djoco\AppData\Local\Temp\ipykernel_2676\4092999231.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data.append(liTag)
C:\Users\djoco\anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])
C:\Users\djoco\AppData\Local\Temp\ipykernel_2676\4092999231.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Work_Org": work_org, "Type_of_Work": type_of_work, "Address": address, "Phone_Number": phone_number}, ignore_index=True)
C:\Users\djoco\AppData\Local\Temp

C:\Users\djoco\AppData\Local\Temp\ipykernel_2676\4092999231.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data.append(liTag)
C:\Users\djoco\anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])
C:\Users\djoco\AppData\Local\Temp\ipykernel_2676\4092999231.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Work_Org": work_org, "Type_of_Work": type_of_work, "Address": address, "Phone_Number": phone_number, "Website": website, "Email": email}, ignore_index=Tru

In [966]:
data

,Work_Org,Type_of_Work,Address,Phone_Number,Email,Website
0,Sub-Contractors,Thermal and Moisture Protection,"32 Goodmark Place Unit 17, \n Etobicoke, \n...",416-679-8911,NaN,NaN
1,General Contractors,General Building,"27 Nashville Ave, Suite 1, \n York, \n O...",416-825-9037\nFax (888) 521-3334,NaN,NaN
2,General Contractors,Civil,"1160 Waterdown Rd Unit 1, \n Burlington, \n...",905-633-6938\nFax 905-633-7132,NaN,NaN
3,Sub-Contractors,"Heating, Ventilating, and Air Conditioning (HV...","2285 St-Laurent Blvd. Unit B19, \n Ottawa, ...",3vmechanical.ca/,info@3vmechanical.ca,613-832-4595\nFax 613-260-3540
4,Sub-Contractors,Exterior Improvements,"5104 Jones Baseline, \n Guelph, \n Ontar...",519-221-8688,NaN,NaN
5,General Contractors,General Building,"34 allanhurst Crescent, \n Brampton, \n ...",416-732-1966,NaN,NaN
6,Construction Software,"3760 14th Avenue, \n Markham, \n Ontario...",canada.constructconnect.com,800-465-6475\nFax 800-947-7966,NaN,customerrelationscanada@constructconnect.com
7,Sub-Contractors,Electronic Safety and Security,"360 Carlingview Dr, \n Etobicoke, \n Ont...",www.360asc.com/,info@360asc.com,416-798-2228\nFax (416) 798-3962
8,"Site Services, Sub-Contractors",Earthwork,"815 Great Northern Road, \n Sault Ste Marie...",705-253-3569\nFax 705-253-8807,NaN,shanecorbett@shaw.ca
9,"General Contractors, Site Services",General Building,"2355 Royal Windsor Dr, Unit 6, \n Mississau...",www.aandocontracting.com,info@aandocontracting.com,905-828-6868\nFax 905-855-1744


In [1015]:
new_data = data
new_data = pd.DataFrame(0.0, index=np.arange(len(data)), columns=["Work_Org", "Type_of_Work", "Address", "Phone_Number", "Website", "Email"])

x = len(data)
for i in range(0,x,1):
    for j in range(0,6,1):
        
        # Find Address - the address has multiple \n tags
        x = len(re.findall('\n', str(data.iloc[i,j])))
        if x > 1:
            new_data.iloc[i,2] = data.iloc[i,j]
        
#         FIX THIS
#         # Find Website
#         if '.com' or '.ca' and not '@' in str(data.iloc[i,j]):
#             new_data.iloc[i,4] = data.iloc[i,j]
            
        # Find Email - emails have the @ sign
        if '@' in str(data.iloc[i,j]):
            new_data.iloc[i,5] = data.iloc[i,j]
    
    


new_data
# new_data.columns = ["Work_Org", "Type_of_Work", "Address", "Phone_Number", "Website", "Email"]

,Work_Org,Type_of_Work,Address,Phone_Number,Website,Email
0,0.0,0.0,"32 Goodmark Place Unit 17, \n Etobicoke, \n...",0.0,NaN,0.0
1,0.0,0.0,"27 Nashville Ave, Suite 1, \n York, \n O...",0.0,NaN,0.0
2,0.0,0.0,"1160 Waterdown Rd Unit 1, \n Burlington, \n...",0.0,NaN,0.0
3,0.0,0.0,"2285 St-Laurent Blvd. Unit B19, \n Ottawa, ...",0.0,613-832-4595\nFax 613-260-3540,info@3vmechanical.ca
4,0.0,0.0,"5104 Jones Baseline, \n Guelph, \n Ontar...",0.0,NaN,0.0
5,0.0,0.0,"34 allanhurst Crescent, \n Brampton, \n ...",0.0,NaN,0.0
6,0.0,0.0,"3760 14th Avenue, \n Markham, \n Ontario...",0.0,customerrelationscanada@constructconnect.com,customerrelationscanada@constructconnect.com
7,0.0,0.0,"360 Carlingview Dr, \n Etobicoke, \n Ont...",0.0,416-798-2228\nFax (416) 798-3962,info@360asc.com
8,0.0,0.0,"815 Great Northern Road, \n Sault Ste Marie...",0.0,shanecorbett@shaw.ca,shanecorbett@shaw.ca
9,0.0,0.0,"2355 Royal Windsor Dr, Unit 6, \n Mississau...",0.0,905-828-6868\nFax 905-855-1744,info@aandocontracting.com


In [800]:
j = len(data)
for i in range(0,j,1):
    for x in range(0,5,1):
        print(data.iloc[i,x])


Sub-Contractors
Thermal and Moisture Protection
32 Goodmark Place Unit 17, 
    Etobicoke, 
    Ontario 
    M9W 6J4
416-679-8911
nan
General Contractors
General Building
27 Nashville Ave, Suite 1, 
    York, 
    Ontario 
    M6M 1J2
416-825-9037
Fax (888) 521-3334
nan
General Contractors
Civil
1160 Waterdown Rd Unit 1, 
    Burlington, 
    Ontario 
    L7T 4A8
905-633-6938
Fax 905-633-7132
nan
Sub-Contractors
Heating, Ventilating, and Air Conditioning (HVAC), Plumbing
2285 St-Laurent Blvd. Unit B19, 
    Ottawa, 
    Ontario 
    K1G 4Z5
3vmechanical.ca/
info@3vmechanical.ca
Sub-Contractors
Exterior Improvements
5104 Jones Baseline, 
    Guelph, 
    Ontario 
    N1H 6H8
519-221-8688
nan
General Contractors
General Building
34 allanhurst Crescent, 
    Brampton, 
    Ontario 
    L6P 1C6
416-732-1966
nan
Construction Software
3760 14th Avenue, 
    Markham, 
    Ontario 
    L3R 3T7
canada.constructconnect.com
800-465-6475
Fax 800-947-7966
nan
Sub-Contractors
Electronic Safety and S

In [995]:
data.iloc[2,2]

'1160 Waterdown Rd Unit 1, \n    Burlington, \n    Ontario \n    L7T 4A8'

In [1004]:
# # Find Address
# if data.iloc[2,2] > '\n':
#     print('yes')

print(data.iloc[2,2])
if "\n" in str(data.iloc[2,2]):
    print('yes')
else:
    print('no')
    
    
import re
x = len(re.findall('\n', str(data.iloc[2,2])))
x

1160 Waterdown Rd Unit 1, 
    Burlington, 
    Ontario 
    L7T 4A8
yes


3

In [ ]:
# Find Email
if '@' in data.iloc[3,4]:
    print('yes')

In [1014]:
# Find Website
if '.com' or '.ca' and not '@' in data.iloc[3,3]:
    print('yes')


yes


In [815]:
data.iloc[3,3]

'3vmechanical.ca/'

In [832]:
# Phone number
x = data.iloc[3,5]
count = 0
for i in x:
    if i.isdigit():
        count = count +1

if count > 8:
    print('yes')

yes


In [875]:
for index, row in data.iterrows():
        print(row[1])

Thermal and Moisture Protection
General Building
Civil
Heating, Ventilating, and Air Conditioning (HVAC), Plumbing
Exterior Improvements
General Building
3760 14th Avenue, 
    Markham, 
    Ontario 
    L3R 3T7
Electronic Safety and Security
Earthwork
General Building
Civil
General Building
General Building
Heating, Ventilating, and Air Conditioning (HVAC)
Masonry, Openings
General Building
